# TensorFlow计算模型--计算图

## 计算图的使用

TensorFlow的程序一般分为两个阶段：构建计算图和执行计算图。
在Tensorflow程序中，系统会自动维护一个默认的计算图，通过`tf.get_default_graph`函数可以获取默认的计算图

In [11]:
import tensorflow as tf
import numpy as np

a = tf.constant([1.0, 2.0], name='a')
b = tf.constant([2.0, 3.0], name='b')
result = a + b

# 通过a.graph可以查看张量所属的计算图。
# 因为没有特意指定，所以这个计算图应该属于当前默认的计算图。
# 所以的下面的操作应该位true
print(a.graph is tf.get_default_graph())

True


除了使用默认的计算图以外，TensorFlow支持通过`tf.Graph`函数来生成新的计算图。
不同计算图上的张量和运算都不会共享。以下代码示意了在不同计算图上定义和使用变量。

In [1]:
import tensorflow as tf

g1 = tf.Graph()
with g1.as_default():
    # 在计算图g1上定义变量"v"，并设置值为0
    v = tf.get_variable("v", shape=[1], initializer=tf.zeros_initializer())
    
g2 = tf.Graph()
with g2.as_default():
    # 在计算图g2上定义变量"v"，并设置值为1
    v = tf.get_variable("v", shape=[1], initializer=tf.ones_initializer())
    
# 在计算图g1中读取变量"v"的取值
with tf.Session(graph=g1) as sess:
    # 变量初始化
    tf.global_variables_initializer().run()
    
    with tf.variable_scope("", reuse=True):
        # 在计算图g1中，变量"v"的取值应该是0，所以下面的结果应该是[0.]
        print(sess.run(tf.get_variable("v")))
        
# 在计算图g2中读取变量"v"的取值
with tf.Session(graph=g2) as sess:
    # 变量初始化
    tf.global_variables_initializer().run()
    
    with tf.variable_scope("", reuse=True):
        # 在计算图g1中，变量"v"的取值应该是0，所以下面的结果应该是[0.]
        print(sess.run(tf.get_variable("v")))


[0.]
[1.]


上面的代码产生了两个计算图，每个计算图中定义了一个名字为"v"的变量。在计算图中g1中，将v初始化为0；在计算图g2中，将v初始化为1。可以看到当运行不同的计算图时，v的变量值也不一样。

TensorFlow中的计算图不仅仅可以用来隔离张量和计算，它还提供了管理张量和计算的功能。计算图可以用tf.Graph.device函数来指定运行计算的设备。这是TensorFlow使用GPU提供了机制。以下程序可以让加法跑在GPU上。

In [ ]:
import tensorflow as tf

g = tf.Graph() # 建立一个新的计算图
with g.device('/gpu:0'): # 指定运行的设备
    with g.as_default():
        a = tf.get_variable("a", shape=[2], initializer=tf.zeros_initializer())
        b = tf.get_variable("b", shape=[2], initializer=tf.ones_initializer())
        result = a + b;
    
with tf.Session(graph=g) as sess:
    # 变量初始化操作
    tf.global_variables_initializer().run()
    
    with tf.variable_scope("", reuse=True):
        # 在计算图g中，获取变量result
        print(sess.run(result))

[1. 1.]


# tf.constant_initializer()函数的使用

In [3]:
value = [0, 1, 2, 3, 4, 5, 6, 7]
# value = np.array(value)
# value = value.reshape([2, 4])
init = tf.constant_initializer(value)
print('fitting shape:')
with tf.Session():
    x = tf.get_variable('x', shape=[2, 4], initializer=init)
    x.initializer.run() # 初始化变量的操作
    print(x.eval())

fitting shape:
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]]


# 如果在不同的计算图上定义和使用变量
除了使用默认的计算图，Tensorflow支持通过tf.Graph函数来生成新的计算图。不同的计算图上的张量和运算都不会共享.

TensorFlow 中的tf.get_variable变量初始化函数:

|初始化函数|功能|主要参数|
|:---------|:----|:--------|
|tf.constant_initializer | 将变量初始化为给定常数 | 常数的取值 |
|tf.random_normal_initializer | 将变量初始化为满足正态分布的随机值 | 正态分布的均值和标准差 |
|tf.truncated_normal_initializer | 将变量初始化为满足正态分布的随机值，但若随机值偏离平均值超过2个标准差，<br>则这个数会被重新随机 | 正态分布的均值和标准差 |
|tf.random_uniform_initializer | 将变量初始化为满足平均分布的随机值 | 最大、最小值
|tf.uniform_unit_scaling_initializer | 将变量初始化为满足平均分布但不影响输出数量级的随机值 | factor(产生随机值时乘以的系数) |
|tf.zeros_initializer | 将变量初始化为全0 | 无 |
|tf.ones_initializer | 将变量初始化为全1 | 无 |

In [9]:
# 建立新图 g1
g1 = tf.Graph()
with g1.as_default():
    # 在新图g1中定义变量'v'，并初始化为0
    v = tf.get_variable("v", shape=[1], initializer=tf.zeros_initializer) # initialzer是函数，而不是函数的返回值
    
# 建立新图 g2
g2 = tf.Graph()
with g2.as_default():
    # 在新图g1中定义变量'v'，并初始化为1
    v = tf.get_variable("v", shape=[1], initializer=tf.ones_initializer)
    
# 在计算图g1中读取变量"v"的取值
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        #在计算图g1中，变量“v”的取值应该为0，所以下面会输出[0.]
        print(sess.run(tf.get_variable("v")))
        
# 在计算图g2中读取变量"v"的取值
with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        #在计算图g2中，变量“v”的取值应该为1，所以下面会输出[0.]
        print(sess.run(tf.get_variable("v")))


[ 0.]
[ 1.]


# 在计算图中指定运算设备
TensorFlow中的计算图不仅仅可以用来隔离张量和计算。它还提供管理张量和计算的机制。计算图可以通过tf.Graph.device函数来指定运行计算的设备。
这里使用cpu(因为笔记本上没有GPU)，下面的计算将跑在CPU上

In [24]:
g3 = tf.Graph()
with g3.as_default():
    # 指定运算设备
    with g3.device("/cpu:0"):
        a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
        b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
        c = tf.matmul(a, b)
# Create a session with log_device_placement set to True
# 设置log_device_placement=True可以打印出计算操作在哪个设备上完成
with tf.Session(graph=g3, config=tf.ConfigProto(log_device_placement=True)) as sess:
    print sess.run(c) # 为什么没有打印设备信息呢

[[ 22.  28.]
 [ 49.  64.]]


# TensorFlow中维护的集合表
有效的管理Tensorflow程序中的资源也是计算图的一个重要功能。在一个计算图中，可以通过集合来管理(collection)来管理不同类型的资源。
比如通过`tf.add_to_collection`函数可以加入一个或多个集合中，然后通过`tf.get_collection`获取一个集合里面的所有资源。这里的
资源可以是张量、变量或者运行TensorFlow程度所需要的队列资源等等。为了方便使用，TensorFlow也自动管理了一些最常用的集合。

|集合名称	|集合内容|	使用场景|
|:----------|:--------|:--------|
|tf.GraphKeys.VARIABLES	| 所有变量 | 持久化TensorFlow模型 |
|tf.GraphKeys.TRAINABLE_VARIABLES | 可学习的变量(一般指神经网络中的参数) | 模型训练、生成模型可视化内容
|tf.GraphKeys.SUMMARIES	| 日志生成相关的张量 | TensorFlow计算可视化 |
|tf.GraphKeys.QUEUE_RUNNERS	| 处理输入的QueueRunner | 输入处理 |
|tf.GraphKeys.MOVING_AVERAGE_VARIABLES	| 所有计算了滑动平均值的变量	| 计算变量的滑动平均值 |